In [ ]:
from scipy.io import arff
import pandas as pd
from GAN2 import GAN
from Dataset import Dataset


dataset_name = "diabetes"
data = Dataset(f'Assignment 4 files/{dataset_name}.arff')
data.norm(method='MinMaxScaler')
gan = GAN(32, 0.0002, 100, data.norm_df.shape[1])
history = gan.train(data.norm_df, dataset_name, 200+1, 10)


0 [D loss: 0.797738, acc.: 50.00%] [G loss: 0.709322]
10 [D loss: 0.713567, acc.: 54.69%] [G loss: 0.676001]
20 [D loss: 0.710873, acc.: 56.25%] [G loss: 0.822266]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

seed = 17
np.random.seed(seed)

base_dir = 'model/'

#Actual fraud data visualization
model_steps = [ 0, 10, 20, 30, 40, 50, 100, 150, 170, 200]


def plot_gan_generations(data, gan, model_steps, dataset, test_size=100, noise_dim=100):
    z = np.random.normal(size=(test_size, noise_dim))
    real = gan.get_data_batch(train=data.norm_df, batch_size=test_size, seed=seed)
    real_samples = pd.DataFrame(real)
    models = {'GAN': ['GAN', False, gan.generator]}
    fig, ax = plt.subplots(5, 2, figsize=(12, 16))
    # PCA
    scalar = PCA(n_components=2, svd_solver='full')
    scalar.fit(data.norm_df.values)
    X_embedded = scalar.transform(data.norm_df.values)
    for model_step_ix, model_step in enumerate(model_steps):      
        [model_name, with_class, generator_model] = models['GAN']
        generator_model.load_weights( base_dir + dataset+'_generator_model_weights_step_'+str(model_step)+'.h5')
        g_z = generator_model.predict(z)
        g_z_emb = scalar.transform(g_z)
        ax[model_step_ix // 2, model_step_ix % 2].scatter(X_embedded[:,0], X_embedded[:,1], c='b')
        ax[model_step_ix // 2, model_step_ix % 2].scatter(g_z_emb[:, 0], g_z_emb[:, 1], c='r')
        ax[model_step_ix // 2, model_step_ix % 2].set_title(f'model steps: {model_step}')

In [ ]:
plot_gan_generations(data, gan, model_steps, dataset_name)